In [1]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

In [2]:
def generate_text(project_id: str, location: str) -> str:
    # Initialize Vertex AI
    vertexai.init(project=project_id, location=location)
    # Load the model
    multimodal_model = GenerativeModel("gemini-1.0-pro-vision")
    # Query the model
    response = multimodal_model.generate_content(
        [
            # Add an example image
            Part.from_uri(
                "gs://generativeai-downloads/images/scones.jpg", mime_type="image/jpeg"
            ),
            # Add an example query
            "what is shown in this image?",
        ]
    )
    print(response)
    return response.text

In [3]:
generate_text("bigframes-dev", "us-central1")

candidates {
  content {
    role: "model"
    parts {
      text: " The image shows a table with a cup of coffee, a bowl of blueberries, and a plate of blueberry scones. There are also pink flowers on the table."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 265
  candidates_token_count: 32
  total_token_count: 297
}



' The image shows a table with a cup of coffee, a bowl of blueberries, and a plate of blueberry scones. There are also pink flowers on the table.'

In [4]:
import bigframes.pandas as bpd
import pandas as pd

In [5]:
b = bpd.from_glob_path("gs://garrettwu_bucket/images/*")
b

/usr/local/google/home/garrettwu/src/bigframes/bigframes/session/__init__.py:1710: UserWarning: No explicit location is set, so using location US for the session.
  return Session(context)


,uri
0,gs://garrettwu_bucket/images/images.jpeg
1,gs://garrettwu_bucket/images/images (1).jpeg
2,gs://garrettwu_bucket/images/download.jpeg


In [6]:
df = b
df["question"] = ["what animals are in the image?", "How many cats are there in the image?", "How does the cat feel?"]

In [7]:
df

,uri,question
0,gs://garrettwu_bucket/images/images.jpeg,what animals are in the image?
1,gs://garrettwu_bucket/images/images (1).jpeg,How many cats are there in the image?
2,gs://garrettwu_bucket/images/download.jpeg,How does the cat feel?


In [8]:
from bigframes.ml import vision

In [9]:
model = vision.GeminiMultimodalTextGenerator()

In [10]:
model.predict(df)

col_7    gs://garrettwu_bucket/images/images.jpeg
col_9              what animals are in the image?
Name: 0, dtype: object
col_7    gs://garrettwu_bucket/images/images (1).jpeg
col_9           How many cats are there in the image?
Name: 1, dtype: object
col_7    gs://garrettwu_bucket/images/download.jpeg
col_9                        How does the cat feel?
Name: 2, dtype: object


,uri,question,multimodal_result
0,gs://garrettwu_bucket/images/images.jpeg,what animals are in the image?,The image contains four kittens.
1,gs://garrettwu_bucket/images/images (1).jpeg,How many cats are there in the image?,There are two cats in the image. The first ca...
2,gs://garrettwu_bucket/images/download.jpeg,How does the cat feel?,The cat is feeling playful and wants to inter...
